# Instructions
### 1) hit the white play button below 
### 2) grab yourself a coffee 🍹 (10min wait) 
### 3) scroll all the way to bottom of output and open link "Running on public URL: https://xxxxxxxxx.gradio.live" 
### 4) there are many parameters, read here what they mean: https://github.com/lunarring/latentblending/blob/main/parameters.md
👇 (start here, move cursor below finger and play button will appear)

In [ ]:
#@title
# installs
!pip install open-clip-torch
!pip install omegaconf
!pip install fastcore -U
!pip install Pillow
!pip install ffmpeg-python
!pip install einops
!pip install gradio

import os, sys
from subprocess import getoutput

# Xformers
os.system("pip install --extra-index-url https://download.pytorch.org/whl/cu113 torch torchvision==0.13.1+cu113")
os.system("pip install triton==2.0.0.dev20220701")
gpu_info = getoutput('nvidia-smi')
if("A10G" in gpu_info):
    os.system(f"pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.15/xformers-0.0.15.dev0+4c06c79.d20221205-cp38-cp38-linux_x86_64.whl")
elif("T4" in gpu_info):
    os.system(f"pip install -q https://github.com/camenduru/stable-diffusion-webui-colab/releases/download/0.0.15/xformers-0.0.15.dev0+1515f77.d20221130-cp38-cp38-linux_x86_64.whl")

!pip install pytorch_lightning
!pip install transformers

# Get Latent Blending from git / pull 
!git clone https://github.com/lunarring/latentblending
!cd latentblending; git pull; cd ..
sys.path.append("/content/latentblending")

# Imports
import torch
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import warnings
import torch
from tqdm.auto import tqdm
from PIL import Image
import torch
from typing import Callable, List, Optional, Union
from latent_blending import LatentBlending, add_frames_linear_interp, get_time, yml_save, LatentBlending, compare_dicts
from stable_diffusion_holder import StableDiffusionHolder
from gradio_ui import BlendingFrontend
import gradio as gr

torch.set_grad_enabled(False)
torch.backends.cudnn.benchmark = False


#%% First let us spawn a stable diffusion holder
device = "cuda" 

# ckpt download
if not os.path.isfile('v2-1_512-ema-pruned.ckpt'):
  !wget https://huggingface.co/stabilityai/stable-diffusion-2-1-base/resolve/main/v2-1_512-ema-pruned.ckpt v2-1_512-ema-pruned.ckpt

fp_ckpt = "v2-1_512-ema-pruned.ckpt"
fp_config = 'latentblending/configs/v2-inference.yaml'

sdh = StableDiffusionHolder(fp_ckpt, fp_config, device) 

from latent_blending import get_time, yml_save, LatentBlending, add_frames_linear_interp, compare_dicts
from gradio_ui import BlendingFrontend

import gradio as gr

if __name__ == "__main__":    
    
    self = BlendingFrontend(sdh) # Yes this is possible in python and yes it is an awesome trick
    
    with gr.Blocks() as demo:
        with gr.Row():
            prompt1 = gr.Textbox(label="prompt 1")
            prompt2 = gr.Textbox(label="prompt 2")
            negative_prompt = gr.Textbox(label="negative prompt")          
            
        with gr.Row():
            nmb_branches_final = gr.Slider(5, 125, self.nmb_branches_final, step=4, label='nmb trans images', interactive=True) 
            height = gr.Slider(256, 2048, self.height, step=128, label='height', interactive=True)
            width = gr.Slider(256, 2048, self.width, step=128, label='width', interactive=True) 
            
        with gr.Row():
            num_inference_steps = gr.Slider(5, 100, self.num_inference_steps, step=1, label='num_inference_steps', interactive=True)
            branch1_influence = gr.Slider(0.0, 1.0, self.branch1_influence, step=0.01, label='branch1_influence', interactive=True) 
            guidance_scale = gr.Slider(1, 25, self.guidance_scale, step=0.1, label='guidance_scale', interactive=True) 
    
        with gr.Row():
            depth_strength = gr.Slider(0.01, 0.99, self.depth_strength, step=0.01, label='depth_strength', interactive=True) 
            duration = gr.Slider(0.1, 30, self.duration, step=0.1, label='video duration', interactive=True) 
            guidance_scale_mid_damper = gr.Slider(0.01, 2.0, self.guidance_scale_mid_damper, step=0.01, label='guidance_scale_mid_damper', interactive=True) 
            
        with gr.Row():
            seed1 = gr.Number(42, label="seed 1", interactive=True)
            b_newseed1 = gr.Button("randomize seed 1", variant='secondary')
            seed2 = gr.Number(420, label="seed 2", interactive=True)
            b_newseed2 = gr.Button("randomize seed 2", variant='secondary')
        with gr.Row():
            b_compute_transition = gr.Button('compute transition', variant='primary')
        
        with gr.Row():
            img1 = gr.Image(label="1/5")
            img2 = gr.Image(label="2/5")
            img3 = gr.Image(label="3/5")
            img4 = gr.Image(label="4/5")
            img5 = gr.Image(label="5/5")
        
        with gr.Row():
            vid_transition = gr.Video()
        
        # Bind the on-change methods
        depth_strength.change(fn=self.change_depth_strength, inputs=depth_strength)
        num_inference_steps.change(fn=self.change_num_inference_steps, inputs=num_inference_steps)
        nmb_branches_final.change(fn=self.change_nmb_branches_final, inputs=nmb_branches_final)
        
        guidance_scale.change(fn=self.change_guidance_scale, inputs=guidance_scale)
        guidance_scale_mid_damper.change(fn=self.change_guidance_scale_mid_damper, inputs=guidance_scale_mid_damper)
        
        height.change(fn=self.change_height, inputs=height)
        width.change(fn=self.change_width, inputs=width)
        negative_prompt.change(fn=self.change_negative_prompt, inputs=negative_prompt)
        seed1.change(fn=self.change_seed1, inputs=seed1)
        seed2.change(fn=self.change_seed2, inputs=seed2)
        duration.change(fn=self.change_duration, inputs=duration)
        branch1_influence.change(fn=self.change_branch1_influence, inputs=branch1_influence)
    
        b_newseed1.click(self.randomize_seed1, outputs=seed1)
        b_newseed2.click(self.randomize_seed2, outputs=seed2)
        b_compute_transition.click(self.compute_transition, 
                                   inputs=[prompt1, prompt2],
                                   outputs=[img1, img2, img3, img4, img5, vid_transition])
        
    demo.launch(share=self.share, inbrowser=True, inline=False, debug=True)


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 KB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 KB 822.3 kB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for antlr4-python3-runtime: filename=antlr4_python3_runtime-4.9.3-py3-none-any.whl size=144575 sha256=0628fded204cc90e08bd2ad25cdb360e7f5019eb3b6b2daa8dd3ca9b8b58402f
  Stored in directory: /root/.cache/pip/wheels/b1/a3/c2/6df046c09459b73cc9bb6c4401b0be6c47048baf9a1617c485
Successfully built antlr4-python3-runtime


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.6/41.6 KB 5.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 182.4/182.4 KB 22.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 71.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 71.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 KB 7.5 MB/s eta 0:00:00
  Attempting uninstall: pro

/usr/local/lib/python3.8/dist-packages/pytorch_lightning/utilities/distributed.py:258: LightningDeprecationWarning: `pytorch_lightning.utilities.distributed.rank_zero_only` has been deprecated in v1.8.1 and will be removed in v1.10.0. You can import it from `pytorch_lightning.utilities` instead.
  rank_zero_deprecation(


--2023-01-14 14:33:52--  https://huggingface.co/stabilityai/stable-diffusion-2-1-base/resolve/main/v2-1_512-ema-pruned.ckpt
Resolving huggingface.co (huggingface.co)... 3.231.67.228, 54.235.118.239, 2600:1f18:147f:e800:671:b733:ecf3:a585, ...
Connecting to huggingface.co (huggingface.co)|3.231.67.228|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/24/cb/24cbc2f7542236eb613b4f16b6802d7c2bef443e86cf9d076719733866e66c3a/88ecb782561455673c4b78d05093494b9c539fc6bfc08f3a9a4a0dd7b0b10f36?response-content-disposition=attachment%3B%20filename%3D%22v2-1_512-ema-pruned.ckpt%22&Expires=1673960148&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zLzI0L2NiLzI0Y2JjMmY3NTQyMjM2ZWI2MTNiNGYxNmI2ODAyZDdjMmJlZjQ0M2U4NmNmOWQwNzY3MTk3MzM4NjZlNjZjM2EvODhlY2I3ODI1NjE0NTU2NzNjNGI3OGQwNTA5MzQ5NGI5YzUzOWZjNmJmYzA4ZjNhOWE0YTBkZDdiMGIxMGYzNj9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPWF0dGFjaG1lbnQlM0IlMjBmaWxlb

Downloading:   0%|          | 0.00/3.94G [00:00<?, ?B/s]

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://014826da-2448-4ebf.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades (NEW!), check out Spaces: https://huggingface.co/spaces
Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://014826da-2448-4ebf.gradio.live
